# Constraining Types


In [27]:
from typing import Optional, Union, Literal, Annotated, Final, NewType
from dataclasses import dataclass


## Optional type

Consider the function below. Every step in the function can throw None

In [2]:
def dispense_bun(): return
def dispense_frank(): return
def dispense_mustard(): return
def dispense_ketchup(): return
def dispense_hot_dog_to_customer(): return


def create_hot_dog():
    bun = dispense_bun()
    frank = dispense_frank()
    hot_dog = bun.add_frank(frank)
    ketchup = dispense_frank()
    mustard = dispense_mustard()
    hot_dog.add_condiments(ketchup, mustard)
    dispense_hot_dog_to_customer(hot_dog)



To protect from None, we need to code defensively, as follows

In [3]:
def print_error_code(): return

def create_hot_dog():
    bun = dispense_bun()
    if bun is None:
        print_error_code("Bun unavailable. Check for bun")
        return
    
    frank = dispense_frank()
    if frank is None:
        print_error_code("Frank was not properly dispensed")
        return
    
    hot_dog = bun.add_frank(frank)
    if hot_dog is None:
        print_error_code("Hot Dog unavailable. Check for Hot Dog")
        return
    
    ketchup = dispense_ketchup()
    mustard = dispense_mustard()
    if ketchup is None or mustard is None:
        print_error_code("Check for invalid catsup")
        return
    
    hot_dog.add_condiments(ketchup, mustard)
    dispense_hot_dog_to_customer(hot_dog)


To guard against None, you can use Optional types, as follows

In [4]:
maybe_a_string: Optional[str] = "abcdef" # This has a value
maybe_a_string


'abcdef'

In [5]:
maybe_a_string: Optional[str] = None # This is the absence of a value
maybe_a_string

The Optional type is especially useful in return types, as follows

In [6]:
# Create a file dispense_bun.py with the following contents

def are_buns_available():
    return

class Bun:
    def __init__(self) -> None:
        pass
    
    def add_frank(self, x):
        pass

def dispense_bun() -> Bun:
    if not are_buns_available():
        return None
    return Bun()


In [7]:
# Then run mypy against the file. It will warn that the None case was not covered

!mypy dispense_bun.py


Success: no issues found in 1 source file


In [8]:
# To correct the last error, modify the signature of 'dispense_bun()' as follows

def dispense_bun() -> Optional[Bun]:
    if not are_buns_available():
        return None
    return Bun()


In [9]:
# Then run mypy against the file. Now the code will typecheck successfully

!mypy dispense_bun.py


Success: no issues found in 1 source file


The calling code also benefits from the Optional type, as follows


In [10]:
# Create a file hotdog.py with the following contents

from dispense_bun import dispense_bun


def dispense_frank(): return
def dispense_mustard(): return
def dispense_ketchup(): return
def dispense_hot_dog_to_customer(x): return


def create_hot_dog() -> None:
    bun = dispense_bun()
    frank = dispense_frank()
    hot_dog = bun.add_frank(frank)
    ketchup = dispense_frank()
    mustard = dispense_mustard()
    hot_dog.add_condiments(ketchup, mustard)
    dispense_hot_dog_to_customer(hot_dog)


In [20]:
# Then run mypy against the file. It will warn that 
# the None case is not covered in the calling code

!mypy hotdog.py


hotdog_invalid.py:13: error: Item "Bun" of "Optional[Bun]" has no attribute "add_frank"
hotdog_invalid.py:13: error: Item "None" of "Optional[Bun]" has no attribute "add_frank"
Found 2 errors in 1 file (checked 1 source file)


In [ ]:
# To correct the last warning, modify the return type to handle the None case

def print_error_code(x): return

def create_hot_dog() -> None:
    bun = dispense_bun()
    if bun is None:
        print_error_code("Bun could not be dispensed")
        return

    frank = dispense_frank()
    hot_dog = bun.add_frank(frank)
    ketchup = dispense_frank()
    mustard = dispense_mustard()
    hot_dog.add_condiments(ketchup, mustard)
    dispense_hot_dog_to_customer(hot_dog)


In [26]:
# Then run mypy against the file. Now the code will typecheck successfully

!mypy hotdog.py


Success: no issues found in 1 source file


## Union Types

A simple function with only 1 return type

In [32]:
def are_ingredients_available(): return
def order_interrupted(): return

class HotDog:
    pass

class Pretzel:
    pass

def dispense_snack() -> HotDog:
    if not are_ingredients_available():
        raise RuntimeError("Not all ingredients available")
    if order_interrupted():
        raise RuntimeError("Order interrupted")
    return create_hot_dog()


Functions that return more than 1 type can be annotated with Union, as follows

In [33]:
def dispense_hot_dog(): return
def dispense_pretzel(): return

def dispense_snack(user_input: str) -> Union[HotDog, Pretzel]:
    if user_input == "Hot Dog":
        return dispense_hot_dog()
    elif user_input == "Pretzel":
        return dispense_pretzel()
    raise RuntimeError(
        "Should never reach this code, as an invalid input has been entered"
    )


Union types can also detect type errors in the calling code, as follows

In [ ]:
# Create a file union_hotdog.py with the following contents

def get_order(): return

def place_order() -> Optional[HotDog]:
    order = get_order()
    result = dispense_snack(order.name) # This returns Union[HotDog, Pretzel]
    if result is None:
        print_error_code("An error occurred" + result)
        return None
    return result # Return our HotDog

In [34]:
# Then run mypy against the file. It will warn that there's a mismatch
# between the return type of 'result' and the return type of 'place_order()'

!mypy union_hotdog.py


union_hotdog.py:30: error: Incompatible return value type (got "Union[HotDog, Pretzel]", expected "Optional[HotDog]")
Found 1 error in 1 file (checked 1 source file)


In [ ]:
# To correct the last warning, modify the return type to handle the None case

def place_order() -> Optional[Union[HotDog, Pretzel]]:
    order = get_order()
    result = dispense_snack(order.name)
    if result is None:
        print_error_code("An error occurred" + result)
        return None
    return result # Return our HotDog

In [35]:
# Then run mypy against the file. Now the code will typecheck successfully

!mypy union_hotdog.py


Success: no issues found in 1 source file


### Product and Sum Types


As an example of product types, consider the following dataclass

In [38]:
@dataclass
class Snack:
    name: str
    condiments: set[str]
    error_code: int
    disposed_of: bool

Snack("Hotdog", {"Mustard", "Ketchup"}, 5, False)


Snack(name='Hotdog', condiments={'Ketchup', 'Mustard'}, error_code=5, disposed_of=False)

```
Assuming the following about the possible field values of Snack:
• The name can be one of three values: hotdog, pretzel, or veggie burger
• The condiments can be empty, mustard, ketchup, or both.
• There are six error codes (0–5); 0 indicates success).
• disposed_of is only True or False.

The number of values that can be represented in this combination of fields is 144: 
• 3 possible values for name × 
• 3 possible values for condiments × 
• 6 possible values for error_code x
• 2 possible values for disposed_of = 3×4×6×2 = 144
```

But with union types, the number of (possible invalid) states can be reduced drastically

In [39]:
@dataclass
class Error:
    error_code: int
    disposed_of: bool

@dataclass
class Snack:
    name: str
    condiments: set[str]


In [42]:
snack: Union[Snack, Error] = Snack("Hotdog", {"Mustard", "Ketchup"})
snack

Snack(name='Hotdog', condiments={'Ketchup', 'Mustard'})

In [41]:
snack = Error(5, True)
snack

Error(error_code=5, disposed_of=True)

```
In this case, snack can be either:
• a Snack (a name and condiments), with 3 names and 4 list values, so 12 representable states. 
• an Error (a number and a boolean), with 5 values for the error code (the 0 can be removed, since that was only for success) and 2 values for so 10 representable states.  

Since the Union is an either/or construct, the number of representable states is 22.
```

## Literal Types


The last 2 classes can be expressed in terms of the Literal type

In [10]:
# Create a file literals.py with the following contents

# The modified data classes

@dataclass
class Error:
    error_code: Literal[1, 2 ,3, 4, 5]
    disposed_of: bool

@dataclass
class Snack:
    name: Literal["Pretzel", "Hot Dog", "Veggie Burger"]
    condiments: set[Literal["Mustard", "Ketchup"]]

# Trying to instantiate the data classes with wrong values will raise type errors

Error(0, False)
Snack("Invalid", set())
Snack("Pretzel", {"Mustard", "Relish"})


Snack(name='Pretzel', condiments={'Relish', 'Mustard'})

In [11]:
# Then run mypy against the file. It will warn that there are invalid values 
# in the dataclass instantiations

!mypy literals.py


literals.py:17: error: Argument 1 to "Error" has incompatible type "Literal[0]"; expected "Literal[1, 2, 3, 4, 5]"
literals.py:18: error: Argument 1 to "Snack" has incompatible type "Literal['Invalid']"; expected "Literal['Pretzel', 'Hot Dog', 'Veggie Burger']"
literals.py:19: error: Argument 2 to <set> has incompatible type "Literal['Relish']"; expected "Literal['Mustard', 'Ketchup']"
Found 3 errors in 1 file (checked 1 source file)


## Annotated Types


With Annotated, you can specify arbitrary metadata alongside the type annotation


In [16]:
# ValueRange and MatchesRegex are not built-in types; they are arbitrary expressions

class ValueRange:
    def __init__(self, start, stop):
        pass

class MatchesRegex:
    def __init__(self, pattern):
        pass

# Annotated is best served as a communication method, 
# since there are no tools that will typecheck this for you. 

x: Annotated[int, ValueRange(3,5)]
y: Annotated[str, MatchesRegex('[0-9]{4}')]



## NewType


In [17]:
class HotDog:
    # ... snip hot dog class implementation ...
    pass

def dispense_to_customer(hot_dog: HotDog):
    # note, this should only accept ready-to-serve hot dogs.
    # ...
    return

In [23]:
# NewType is useful when you want some of your functions to operate only on valid hot dogs. 

class HotDog:
    ''' Used to represent an unservable hot dog'''
    # ... snip hot dog class implementation ...
    pass

# Valid hot dogs are created with NewType

ReadyToServeHotDog = NewType("ReadyToServeHotDog", HotDog)

def dispense_to_customer(hot_dog: ReadyToServeHotDog):
    # ...
    return


In [24]:
# It is important to notify users that the only way to create your new type 
# is through a set of 'blessed' functions. 

def prepare_for_serving(hot_dog: HotDog) -> ReadyToServeHotDog:
    assert not hot_dog.is_plated(), "Hot dog should not already be plated"
    hot_dog.put_on_plate()
    hot_dog.add_napkins()
    return ReadyToServeHotDog(hot_dog)

def serve_to_customer(x: ReadyToServeHotDog):
    return

def make_snack():
    hotdog = HotDog()
    valid_hotdog = prepare_for_serving(hotdog)
    serve_to_customer(valid_hotdog)


In [25]:
# Unfortunately, Python has no great way of telling users this, other than a comment.

ReadyToServeHotDog = NewType("ReadyToServeHotDog", HotDog) # NOTE: Only create ReadyToServeHotDog using prepare_for_serving method.


## Type Aliases


In [26]:
# A type alias provides another name for a type and 
# is completely interchangeable with the old type

class User:
    pass

IdOrName = Union[str, int]

IDOrNameLookup =  Union[
    dict[int, User], list[dict[str, User]]
]


## Final types

Final types indicate to a typechecker that a variable cannot be bound to another value.

In [28]:
# Create a file final.py with the following contents

VENDOR_NAME: Final = "Viafore's Auto-Dog"

def display_vendor_information():
    vendor_info = "Auto-Dog v1.0"
    VENDOR_NAME += VENDOR_NAME # This code should be vendor_info += VENDOR_NAME
    print(vendor_info)


In [29]:
# Then run mypy against the file. It will warn that 
# a function is trying to modify a final type

!mypy final.py


final.py:8: error: Cannot assign to final name "VENDOR_NAME"
Found 1 error in 1 file (checked 1 source file)
